<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Finetune_Llama_3_2_Vision_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Install and upgrade UnSloth library for optimized model training

- Use `%%capture` to suppress installation output in Jupyter/Colab environments.
- Install the `unsloth` package from PyPI for initial setup.
- Uninstall the existing `unsloth` package to ensure a clean installation.
- Upgrade to the latest version of `unsloth` directly from the GitHub repository.

In [ ]:
%%capture
# The `%%capture` magic in Jupyter/Colab captures output, suppressing it from being displayed.

# Install the `unsloth` package from PyPI
!pip install unsloth

# Uninstall `unsloth` to ensure a clean installation, then install the latest version from GitHub
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


### 2. Load the model


Load the `unsloth/Llama-3.2-11B-Vision-Instruct` model using FastVisionModel.

Enable 4-bit quantization to reduce memory usage.
Utilize UnSloth's gradient checkpointing for efficient training and inference.


In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

### 3. Add LoRA fine-tuning configuration for Llama 3.2 Vision model

- Apply LoRA (Low-Rank Adaptation) to fine-tune the vision and language components of the model.
- Enable fine-tuning for vision layers, language layers, attention modules, and MLP modules.
- Configure LoRA parameters such as rank (`r`), alpha (`lora_alpha`), and dropout (`lora_dropout`).


In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r                          = 16,
    lora_alpha                 = 16,
    lora_dropout               = 0,
    bias                       = "none",
    random_state               = 3407,
    use_rslora                 = False,
    loftq_config               = None,
)

In [ ]:

from datasets import load_dataset

dataset = load_dataset("DataStudio/OCR_handwritting_HAT2023",
                       split="train[0:500]")

dataset

In [ ]:

dataset[45]["text"]

In [ ]:
instruction = """
You are an expert in Optical Character Recognition (OCR). Your task is to accurately extract and transcribe text from images while preserving the original structure, formatting, and readability. Ensure precision in capturing all characters, words, and symbols, minimizing errors and distortions.
"""

def convert_to_conversation(sample):
    """
    Converts a dataset sample into a structured conversation format for OCR fine-tuning.

    Parameters:
    sample (dict): A dictionary containing an image and its corresponding extracted text.

    Returns:
    dict: A structured conversation containing user input (instruction + image)
          and assistant output (extracted text).
    """
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": sample["image"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["text"]}],
        },
    ]
    return {"messages": conversation}

# Convert the entire dataset into conversation format
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

### 6. Enable inference on Llama 3.2 Vision model for generating product descriptions

- Prepare the model for inference using `FastVisionModel.for_inference`.
- Generate a product description by processing an image and instruction using the Vision-Language Model.
- Use a streaming approach to display generated text in real-time.


In [ ]:

FastVisionModel.for_inference(model)  # Enable for inference!

image = dataset[45]["image"]

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": instruction},
        ],
    }
]
input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

### 7. Set up SFTTrainer for fine-tuning Llama 3.2 Vision model on product descriptions

- Enable the model for training using `FastVisionModel.for_training`.
- Configure the `SFTTrainer` from the `trl` library for supervised fine-tuning (SFT) with UnSloth optimizations.
- Use `UnslothVisionDataCollator` to handle multi-modal data efficiently during training.


In [ ]:

from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)  # Enable for training!

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  # Must use!
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # For Weights and Biases
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=2048,
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:

model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    model="lora_model",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

FastVisionModel.for_inference(model)  # Enable for inference!

image = dataset[40]["image"]

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": instruction},
        ],
    }
]
input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)